# Documentation
> 201025: This notebook generate embedding vectors for pfam_motors, df_dev, and motor_toolkit from the models that currently finished training:
    - lstm5  
          - evotune_lstm_5_balanced.pt  
          - evotune_lstm_5_balanced_target.pt  
          - mini_lstm_5_balanced.pt   
          - mini_lstm_5_balanced_target.pt   
    - transformer_encoder   
          - evotune_seq2seq_encoder_balanced.pt   
          - evotune_seq2seq_encoder_balanced_target.pt  
          - mini_seq2seq_encoder_balanced.pt  
          - mini_seq2seq_encoder_balanced_target.pt  
    - seq2seq_attention_mini  
          - transformer_encoder_201025.pt  
          - evotune_transformerencoder_balanced.pt  
          - evotune_transformerencoder_balanced_target.pt  
          - mini_evotune_transformerencoder_balanced.pt  
          - mini_evotune_transformerencoder_balanced_target.pt  
        

- output for motor_toolkit,pfamA_random, and pfamA_motors


In [1]:
import math
import torch.nn as nn
import argparse
import random
import warnings
import numpy as np
import torch
import torch.nn.functional as F
from torch import optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.autograd import Variable
import itertools
import pandas as pd
from torch.nn import TransformerEncoder, TransformerEncoderLayer
import math

seed = 7
torch.manual_seed(seed)
np.random.seed(seed)

In [2]:
pfamA_motors = pd.read_csv("../../data/pfamA_motors.csv")
pfamA_random = pd.read_csv("../../data/pfamA_random_201027.csv")
motor_toolkit = pd.read_csv("../../data/motor_tookits.csv")

pfamA_motors_balanced = pfamA_motors.groupby('clan').apply(lambda _df: _df.sample(4500,random_state=1))
pfamA_motors_balanced = pfamA_motors_balanced.apply(lambda x: x.reset_index(drop = True))

pfamA_target_name = ["PF00349","PF00022","PF03727","PF06723",\
                       "PF14450","PF03953","PF12327","PF00091","PF10644",\
                      "PF13809","PF14881","PF00063","PF00225","PF03028"]

pfamA_target = pfamA_motors.loc[pfamA_motors["pfamA_acc"].isin(pfamA_target_name),:]


aminoacid_list = [
    'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
    'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'
]
clan_list = ["actin_like","tubulin_c","tubulin_binding","p_loop_gtpase"]
        
aa_to_ix = dict(zip(aminoacid_list, np.arange(1, 21)))
clan_to_ix = dict(zip(clan_list, np.arange(0, 4)))

def word_to_index(seq,to_ix):
    "Returns a list of indices (integers) from a list of words."
    return [to_ix.get(word, 0) for word in seq]

ix_to_aa = dict(zip(np.arange(1, 21), aminoacid_list))
ix_to_clan = dict(zip(np.arange(0, 4), clan_list))

def index_to_word(ixs,ix_to): 
    "Returns a list of words, given a list of their corresponding indices."
    return [ix_to.get(ix, 'X') for ix in ixs]

def prepare_sequence(seq):
    idxs = word_to_index(seq[0:-1],aa_to_ix)
    return torch.tensor(idxs, dtype=torch.long)

def prepare_labels(seq):
    idxs = word_to_index(seq[1:],aa_to_ix)
    return torch.tensor(idxs, dtype=torch.long)

def prepare_eval(seq):
    idxs = word_to_index(seq[:],aa_to_ix)
    return torch.tensor(idxs, dtype=torch.long)

prepare_labels('YCHXXXXX')

# set device
device  = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [12]:
class PositionalEncoding(nn.Module):
    """
    PositionalEncoding module injects some information about the relative or absolute position of
    the tokens in the sequence. The positional encodings have the same dimension as the embeddings 
    so that the two can be summed. Here, we use sine and cosine functions of different frequencies.
    """
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        # print("self.pe.size() :", self.pe[:x.size(0),:,:].size())
        
#         pe[:, 0::2] = torch.sin(position * div_term)
#         pe[:, 1::2] = torch.cos(position * div_term)
#         pe = pe.unsqueeze(0)
        
        self.register_buffer('pe', pe)

    def forward(self, x):
      
#         x = x + self.pe[:x.size(0), :]
        x = x.unsqueeze(0).transpose(0, 1)
        # print("x.size() : ", x.size())
        # print("self.pe.size() :", self.pe[:x.size(0),:,:].size())
        x = torch.add(x ,Variable(self.pe[:x.size(0),:,:], requires_grad=False))
        return self.dropout(x)

class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp, ntoken)

        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        # if self.src_mask is None or self.src_mask.size(0) != src.size(0):
        #     device = src.device
        #     mask = self._generate_square_subsequent_mask(src.size(0)).to(device)
        #     self.src_mask = mask
#         print("src.device: ", src.device)
        src = self.encoder(src) * math.sqrt(self.ninp)
        # print("self.encoder(src) size: ", src.size())
        src = self.pos_encoder(src)
        # print("elf.pos_encoder(src) size: ", src.size())
        output_encoded = self.transformer_encoder(src, self.src_mask)
        output_encoded = self.transformer_encoder(src)
        # print("output_encoded size: ", output_encoded.size())
        output = self.decoder(output_encoded)
        return output,output_encoded

In [14]:
ntokens = len(aminoacid_list) + 1 # the size of vocabulary
emsize = 12 # embedding dimension
nhid = 100 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 6 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 12 # the number of heads in the multiheadattention models
dropout = 0.1 # the dropout value
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout).to(device)
model.eval()

TransformerModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=12, out_features=12, bias=True)
        )
        (linear1): Linear(in_features=12, out_features=100, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=100, out_features=12, bias=True)
        (norm1): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((12,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=12, out_features=12, bias=True)
        )
        (linear1): Linear(in_features=12, out_features=100, bias=T

In [24]:
def generate_embedding_transformer(model,dict_file,dat,dat_name,out_path,out_dir,seq_col):
    # initialize network
    
    model.load_state_dict(torch.load(dict_file))

    print("loaded dict file for weights " + dict_file)
    print("output embedding for " + dat_name)
    
    hn_vector = []
    print_every = 1000
    for epoch in np.arange(0, dat.shape[0]):   
        with torch.no_grad():
            seq = dat.iloc[epoch, seq_col]
            if len(seq) > 5000:
                continue
            sentence_in = prepare_eval(seq)
            sentence_in = sentence_in.to(device = device)
            _,hn = model(sentence_in)
            hn = hn.sum(dim = 0).cpu().detach().numpy()
            hn_vector.append(hn)
            
        if epoch % print_every == 0:
            print(f"At Epoch: %.2f"% epoch)
            print(seq)
    hn_vector = np.array(hn_vector)
    hn_vector = np.squeeze(hn_vector, axis=1)

    print(hn_vector.shape)
    print(out_dir+dat_name+"_"+out_path)
    np.save(out_dir+dat_name+"_"+out_path, hn_vector)
    return 



In [25]:
dict_files = ["evotune_transformer_encoder_mlm_balanced_target.pt","evotune_transformer_encoder_mlm_balanced.pt","evotune_transformerencoder_balanced_target.pt",\
              "evotune_transformerencoder_balanced.pt","mini_transformer_encoder_mlm_balanced_target.pt","mini_transformer_encoder_mlm_balanced.pt",\
              "mini_transformerencoder_balanced_target.pt","mini_transformerencoder_balanced.pt","transformer_encoder_201025.pt","transformer_encoder_mlm_201025.pt"]
dict_files  = ["../../data/201025/"+dict_file for dict_file in dict_files]
dict_files
# "../data/hn_lstm5_motortoolkit.npy"


['../../data/201025/evotune_transformer_encoder_mlm_balanced_target.pt',
 '../../data/201025/evotune_transformer_encoder_mlm_balanced.pt',
 '../../data/201025/evotune_transformerencoder_balanced_target.pt',
 '../../data/201025/evotune_transformerencoder_balanced.pt',
 '../../data/201025/mini_transformer_encoder_mlm_balanced_target.pt',
 '../../data/201025/mini_transformer_encoder_mlm_balanced.pt',
 '../../data/201025/mini_transformerencoder_balanced_target.pt',
 '../../data/201025/mini_transformerencoder_balanced.pt',
 '../../data/201025/transformer_encoder_201025.pt',
 '../../data/201025/transformer_encoder_mlm_201025.pt']

In [26]:
out_paths = ["mlm_evotune_balanced_target.npy","mlm_evotune_balanced.npy","evotune_balanced_target.npy",\
             "evotune_balanced.npy", "mlm_mini_balanced_target.npy","mlm_mini_balanced.npy",\
             "mini_balanced_target.npy", "mini_balanced.npy", "raw.npy","mlm_raw.npy"]

In [27]:
out_dir = "../../out/201027/embedding/transformer_encoder/"
out_paths

['mlm_evotune_balanced_target.npy',
 'mlm_evotune_balanced.npy',
 'evotune_balanced_target.npy',
 'evotune_balanced.npy',
 'mlm_mini_balanced_target.npy',
 'mlm_mini_balanced.npy',
 'mini_balanced_target.npy',
 'mini_balanced.npy',
 'raw.npy',
 'mlm_raw.npy']

In [28]:
len(dict_files)==len(out_paths)

True

In [29]:
pfamA_target.iloc[1,3]

'NSGTGFSKLGFAGNDSPSFVFPTAIATKGPAAGGGGSGSGRPAVGNKPSFLTGGAGPASNHLSSKRGTEDLDFFIGDEATSAAAGPGKLARRRNAQATANHLGYGLHYPIRHGQIENWDHMERFWSNSIFKYLRVEPEDHYFLLTEPPLNPPENRENTAEIFFESFNCAGMYIAVQAVLALAASWTSSKVTDRSLTGTVIDSGDGVTHVIPVAEGYVIGSSIKSIPIAGRDITYFVQSLLRDRGEADSSLKTAQEIKESYCYVCPDIVKEFAKYDRDRSRFLKHTVSLPGGRQVGVDVGYERFLAPEIFFNPEIYSSDFLTPLPVVVDGVIQQSPIDVRRGLYKNIVLSGGSTLYKDFGRRLQRDIKLMVDDRIRASELRSGGARSGGLDVQVISHKRQRHGPWFGGSLLGQTPEFRSYCHTKAEYQEYGPSIVRRFQ'

In [30]:
data = [pfamA_motors_balanced,pfamA_target,pfamA_random,motor_toolkit]
data_names = ["pfamA_motors_balanced", "pfamA_target" , "pfamA_random", "motor_toolkit"]
seq_cols = [3,3,2,7]

In [34]:
for i in range(len(dict_files)):
    dict_file = dict_files[i]
    out_path = out_paths[i]
    for i in range(len(data)):
        dat = data[i]
        dat_name = data_names[i]
        seq_col = seq_cols[i]
        generate_embedding_transformer(model,dict_file,dat,dat_name,out_path,out_dir,seq_col)

loaded dict file for weights ../../data/201025/evotune_transformer_encoder_mlm_balanced_target.pt
output embedding for pfamA_motors_balanced
At Epoch: 0.00
HQDNVHARSLMGLVRNVFEQAGLEKTALDAVAVSSGPGSYTGLRIGVSVAKGLAYALDKPVIGVGTLEALAFRAIPFSDSTDTIIPMLDARRMEVYALVMDGLGDTLISPQPFILEDNPFMEYLEKGKVFFLGDGVPKSKEILSHPNSRFVPLFNSSQSIGELAYKKFLKADFESLAYFEPNYIKEFRI
At Epoch: 1000.00
LAAEARGDRAEAARILGAGAANLVGLLDIDRVVLGGRTVAADEDAYVRGVRAVIADRAARGAGGAHVTVTVADGGDRPVAEGAAQLVLA
At Epoch: 2000.00
ARKIGIDLGTTNLLICVDNKGILVDEPSIITVDATTKKCIAAGLDARDMLGRTPKNMICIRPLKDGVVADFEATDMMLNYFLKKCDLKGMFKKNVILICHPTKITSVEKNAIRDCAYRAGAKKVYLEEEPKIAALGAGLDIGKASGNMVLDIGGGTSDIAVLSLGDIVCSTSIKTAGNKITQDILENVRIQKKMYIGEQTADEIKRRIANALVVKEPETITISGRDVETGLPHSIDINSNEVESYIRSSLQEIVHATKTILEVTPPELAADIVQHGLVLTGGGALLKNLDQLMRNELQIPVYVAENALKCVVDGCTIMLQNL
At Epoch: 3000.00
NSLPSGDQHKAQQLTADYLGALKRHLIDSLKNQLGEHHAKATPLQFILTVPAVWSDAAKEKTLQAAETAGLGQHAPILMISEPEAAATYVLFRKELGGLSTGDTFVVCDAGGGTVDLISYTIEQLEPALQVKEAAPGSGGLCGSTYLNRRFQEFLVTKLGQEEGFDNETVGDAMKKFDEEIKREYSP

At Epoch: 16000.00
EIGLIVGTGTNACYMEELKNVELLDGDEGQMCVNMEWGAFGDNNCLEDITTSFDHDVDTFSINPGKQRYEKMISGMYLGEIVRQILIVLTRRGILFGGKISERLLTRDLFPTRFLSLIESDTLGLVQVRSILTELGLRSTCDDTMLVKEVCTTVSRRAAQLCAAGVAAVVEKMRANRGLDQLKVTVGVDGTLYKLHPHFAGVVQETVKILAPKCDVTFLQSDDGSGRGAALITAV
At Epoch: 17000.00
RLGVIVGTGTNACYMEKLENCELWDGDDQEPRQVIVNTEWGAFGDNGVIDFVRSHYDWEVDEESLNPSHQKFEKMISGMYMGELARRVILRLAREHLIFNGRLSQKMKTAYAFKTKYISEIESDPKGCFDETRKVLAKLDQVGSDDDCQCLKLVVSRVSSRAAHLVSAAIATVLNKMKRPHTTVGVDGSVYRYHPKFHQLMEAKIAELTNPDYKFDLMLSEDGSGRGAALVAAV
At Epoch: 18000.00
QVDIGIDLGTANTLVYLRGHGIVMDEPSVVAVTRGSHTVLNDGAAVGLEAKKMLGKTSYSVDVIRPLREGVIANFPITEAMLRYFISRVKARRMFSQTRVVIAIPFGITHAEMKAVYNSTMRAGADKVHLIEETLAAGLGSGLRIDDPTANLVVDIGGGTTGISVISVADIAFGATVRCAGDHMTDAVSDFIRERYKLQIGQQTAEQLKIELGSALPQNEHAAMQIRGQGENGRPATIEVSADDVREALRAPLHKILRGIDWVLENTPPELSADLVDRGILVTGGGALLPRIDDLISDHTGLNVTVADDPLTCVARGAGAYLDTINWQRS
At Epoch: 19000.00
TKDMGIDLGTANTLVYSKGKGIVLREPSVVAINNLTKKPLAVGTEAKQMIGRTPGNIVAIRPLKDGVIADFDITQTMLKKFIEKITNKSAFTSPRIIVCFPSGVTEVERRAIEEATKQAGAR

At Epoch: 48000.00
RLRPLNEKEISRNDALDWECINDTTIIFKNHLPIPERSMYPSAYTFDRVFRSDSTTREVYEAGAKEVALSVVSGINSSIFAYGQTSSGKTFTMSGITEYTMADIYDHIERHKEREFLLKFSAMEIYNESVRDLLSSDTAPLRLLDDPERGTIVEKLTEETLRDWNHLIELLSLCEAQRQIGETALNETSSRSHQILRLTVESSAREFLGNDNSSVLTSTVNFVDLAGSERASQSLSAGTRLKEGCHINRSLLTLGTVIRKLSKGRSGHIPYRDSKLTRILQSSLGGNAKTAIICTMSPARSHVEQSRNTLLFASCAKEV
At Epoch: 49000.00
PSPRPSISQTPIRTKLQLVDLAGSESVGMSGVSGAALWENSCINRSLSALSDVLGALAEQRPHVPYRNSKLTHLLQDSIGGDAKLLVMLCVSPTQRFLTESLQSLGFGSRARQI
At Epoch: 50000.00
KEYTFDGVFDQESNQKEVYEDVGKPVLKDVLQGYNGSILAYGQTGAGKTHSLLNSGMGVDGKPDPKQAGLLPRLVAALFVHVGADVKHVYTVEASMLQIYNEQVDCLLGDDREKAQGLQVTGKSEVKGLVWHKCKTPNELLQCFQKGRMNLVYAETKMNKSSSRSHAVFQIKVSKRPRALDKTGTKGGKVEMKATFGKLTVVDLAGSERIKKSGVTGTQLKEATNINSSLLSFGNIVQALAEKKKFIPYRDSKLTRILEDSVGGNCKTSLLVCCSPSAESSDETVSTLEFASRAARI
At Epoch: 51000.00
VEDMATLAQLHEGSIMHNLHIRYKKDNIYTYIGSILVSVNPYKSISGLYDITSMEQYSSYHLGERPPHIFAIANECYHCLWKRNDNQCVLISGESGAGKTESTKLILKYLSAMSQHSLDVTAKENVSYVEQAILESSPIMEVFGNAKTIHNNNSSRFGKFIKLNFCQKGNIQGGRIIDYLLEKNRVVRQNP

(3235, 12)
../../out/201027/embedding/transformer_encoder/motor_toolkit_mlm_evotune_balanced_target.npy
loaded dict file for weights ../../data/201025/evotune_transformer_encoder_mlm_balanced.pt
output embedding for pfamA_motors_balanced
At Epoch: 0.00
HQDNVHARSLMGLVRNVFEQAGLEKTALDAVAVSSGPGSYTGLRIGVSVAKGLAYALDKPVIGVGTLEALAFRAIPFSDSTDTIIPMLDARRMEVYALVMDGLGDTLISPQPFILEDNPFMEYLEKGKVFFLGDGVPKSKEILSHPNSRFVPLFNSSQSIGELAYKKFLKADFESLAYFEPNYIKEFRI
At Epoch: 1000.00
LAAEARGDRAEAARILGAGAANLVGLLDIDRVVLGGRTVAADEDAYVRGVRAVIADRAARGAGGAHVTVTVADGGDRPVAEGAAQLVLA
At Epoch: 2000.00
ARKIGIDLGTTNLLICVDNKGILVDEPSIITVDATTKKCIAAGLDARDMLGRTPKNMICIRPLKDGVVADFEATDMMLNYFLKKCDLKGMFKKNVILICHPTKITSVEKNAIRDCAYRAGAKKVYLEEEPKIAALGAGLDIGKASGNMVLDIGGGTSDIAVLSLGDIVCSTSIKTAGNKITQDILENVRIQKKMYIGEQTADEIKRRIANALVVKEPETITISGRDVETGLPHSIDINSNEVESYIRSSLQEIVHATKTILEVTPPELAADIVQHGLVLTGGGALLKNLDQLMRNELQIPVYVAENALKCVVDGCTIMLQNL
At Epoch: 3000.00
NSLPSGDQHKAQQLTADYLGALKRHLIDSLKNQLGEHHAKATPLQFILTVPAVWSDAAKEKTLQAAETAGLGQHAPILMISEPEAAATYV

At Epoch: 16000.00
EIGLIVGTGTNACYMEELKNVELLDGDEGQMCVNMEWGAFGDNNCLEDITTSFDHDVDTFSINPGKQRYEKMISGMYLGEIVRQILIVLTRRGILFGGKISERLLTRDLFPTRFLSLIESDTLGLVQVRSILTELGLRSTCDDTMLVKEVCTTVSRRAAQLCAAGVAAVVEKMRANRGLDQLKVTVGVDGTLYKLHPHFAGVVQETVKILAPKCDVTFLQSDDGSGRGAALITAV
At Epoch: 17000.00
RLGVIVGTGTNACYMEKLENCELWDGDDQEPRQVIVNTEWGAFGDNGVIDFVRSHYDWEVDEESLNPSHQKFEKMISGMYMGELARRVILRLAREHLIFNGRLSQKMKTAYAFKTKYISEIESDPKGCFDETRKVLAKLDQVGSDDDCQCLKLVVSRVSSRAAHLVSAAIATVLNKMKRPHTTVGVDGSVYRYHPKFHQLMEAKIAELTNPDYKFDLMLSEDGSGRGAALVAAV
At Epoch: 18000.00
QVDIGIDLGTANTLVYLRGHGIVMDEPSVVAVTRGSHTVLNDGAAVGLEAKKMLGKTSYSVDVIRPLREGVIANFPITEAMLRYFISRVKARRMFSQTRVVIAIPFGITHAEMKAVYNSTMRAGADKVHLIEETLAAGLGSGLRIDDPTANLVVDIGGGTTGISVISVADIAFGATVRCAGDHMTDAVSDFIRERYKLQIGQQTAEQLKIELGSALPQNEHAAMQIRGQGENGRPATIEVSADDVREALRAPLHKILRGIDWVLENTPPELSADLVDRGILVTGGGALLPRIDDLISDHTGLNVTVADDPLTCVARGAGAYLDTINWQRS
At Epoch: 19000.00
TKDMGIDLGTANTLVYSKGKGIVLREPSVVAINNLTKKPLAVGTEAKQMIGRTPGNIVAIRPLKDGVIADFDITQTMLKKFIEKITNKSAFTSPRIIVCFPSGVTEVERRAIEEATKQAGAR

...

At Epoch: 17000.00
RLGVIVGTGTNACYMEKLENCELWDGDDQEPRQVIVNTEWGAFGDNGVIDFVRSHYDWEVDEESLNPSHQKFEKMISGMYMGELARRVILRLAREHLIFNGRLSQKMKTAYAFKTKYISEIESDPKGCFDETRKVLAKLDQVGSDDDCQCLKLVVSRVSSRAAHLVSAAIATVLNKMKRPHTTVGVDGSVYRYHPKFHQLMEAKIAELTNPDYKFDLMLSEDGSGRGAALVAAV
At Epoch: 18000.00
QVDIGIDLGTANTLVYLRGHGIVMDEPSVVAVTRGSHTVLNDGAAVGLEAKKMLGKTSYSVDVIRPLREGVIANFPITEAMLRYFISRVKARRMFSQTRVVIAIPFGITHAEMKAVYNSTMRAGADKVHLIEETLAAGLGSGLRIDDPTANLVVDIGGGTTGISVISVADIAFGATVRCAGDHMTDAVSDFIRERYKLQIGQQTAEQLKIELGSALPQNEHAAMQIRGQGENGRPATIEVSADDVREALRAPLHKILRGIDWVLENTPPELSADLVDRGILVTGGGALLPRIDDLISDHTGLNVTVADDPLTCVARGAGAYLDTINWQRS
At Epoch: 19000.00
TKDMGIDLGTANTLVYSKGKGIVLREPSVVAINNLTKKPLAVGTEAKQMIGRTPGNIVAIRPLKDGVIADFDITQTMLKKFIEKITNKSAFTSPRIIVCFPSGVTEVERRAIEEATKQAGAREVVLMEEPMAAAIGAGLPVDEPTGSMIVDIGGGTTEVAIISLGGIVTSKSLRIAGDELDQAIIGYIKREYNLMIGERTSEQIKMEIGSAFKADEFEEEASMEIKGRDLISGLPKTVVVTESQIREALKEPVAAIIEAIKTTLEKTPPELAADIMDKGIMLAGGGALLKGLDALINHETHMPVHIAESPLDCVALGAGKALDKFDLIRQ
At Epoch: 20000.00
KSDIGIDLGTASVLVYIKGKGVVIQEP

At Epoch: 49000.00
PSPRPSISQTPIRTKLQLVDLAGSESVGMSGVSGAALWENSCINRSLSALSDVLGALAEQRPHVPYRNSKLTHLLQDSIGGDAKLLVMLCVSPTQRFLTESLQSLGFGSRARQI
At Epoch: 50000.00
KEYTFDGVFDQESNQKEVYEDVGKPVLKDVLQGYNGSILAYGQTGAGKTHSLLNSGMGVDGKPDPKQAGLLPRLVAALFVHVGADVKHVYTVEASMLQIYNEQVDCLLGDDREKAQGLQVTGKSEVKGLVWHKCKTPNELLQCFQKGRMNLVYAETKMNKSSSRSHAVFQIKVSKRPRALDKTGTKGGKVEMKATFGKLTVVDLAGSERIKKSGVTGTQLKEATNINSSLLSFGNIVQALAEKKKFIPYRDSKLTRILEDSVGGNCKTSLLVCCSPSAESSDETVSTLEFASRAARI
At Epoch: 51000.00
VEDMATLAQLHEGSIMHNLHIRYKKDNIYTYIGSILVSVNPYKSISGLYDITSMEQYSSYHLGERPPHIFAIANECYHCLWKRNDNQCVLISGESGAGKTESTKLILKYLSAMSQHSLDVTAKENVSYVEQAILESSPIMEVFGNAKTIHNNNSSRFGKFIKLNFCQKGNIQGGRIIDYLLEKNRVVRQNPGERNYHIFYALLAGTDEAQKEMFYLSEKENYYYLRQFGCIVDNAIDDQRTFQEVMTAMRVMKFSSEEILEVLKLLAGVLHLGNIEFVIAGGAQVSSKNALGRAAELLGLDSMKFTEVLTHRSMILRGEEISTPLTVEQGIDSRDSMAMALYSQCFSWIIKKINNRIKGKEDYRSVGVLDIFGFENFEVNRFEQFNINYANEKLQEYFNKHIFSLEQLEYNRDGLIWEDIDWMDNGECLDLIEKKMGILALINEESHFPKGTDDTLLAKLHSHHSKNPFYVKPRVLDHYFGVKHYAGEVLYHVKGILEKNRDTFRCDVLNLLCESRLDFIYDLFEHASSK

At Epoch: 1000.00
LAAEARGDRAEAARILGAGAANLVGLLDIDRVVLGGRTVAADEDAYVRGVRAVIADRAARGAGGAHVTVTVADGGDRPVAEGAAQLVLA
At Epoch: 2000.00
ARKIGIDLGTTNLLICVDNKGILVDEPSIITVDATTKKCIAAGLDARDMLGRTPKNMICIRPLKDGVVADFEATDMMLNYFLKKCDLKGMFKKNVILICHPTKITSVEKNAIRDCAYRAGAKKVYLEEEPKIAALGAGLDIGKASGNMVLDIGGGTSDIAVLSLGDIVCSTSIKTAGNKITQDILENVRIQKKMYIGEQTADEIKRRIANALVVKEPETITISGRDVETGLPHSIDINSNEVESYIRSSLQEIVHATKTILEVTPPELAADIVQHGLVLTGGGALLKNLDQLMRNELQIPVYVAENALKCVVDGCTIMLQNL
At Epoch: 3000.00
NSLPSGDQHKAQQLTADYLGALKRHLIDSLKNQLGEHHAKATPLQFILTVPAVWSDAAKEKTLQAAETAGLGQHAPILMISEPEAAATYVLFRKELGGLSTGDTFVVCDAGGGTVDLISYTIEQLEPALQVKEAAPGSGGLCGSTYLNRRFQEFLVTKLGQEEGFDNETVGDAMKKFDEEIKREYSPNVPNPNYWVPVPGLATNPRLGIRRNKMTLPPDDVREILKPVIDEVVQLVRKQIQSTEREVKAVLLVGGFGGSQYLLERLKETVTKATVILQ
At Epoch: 4000.00
HIAVDIGGSLAKLVYFSRDPTSKELGGRLNFLKFETARIDECIDFLRKLKLKYEIINGSRPSDLCVMATGGGAFKYYDEIKGALEVEVVREDEMECLIIGLDFFITEIPHEVFTYSQEEPMRFIAARPNIYPYLLVNIGSGVSMVKVSGPRQYERVGGTSLGGGTLWGLLSLLTGARTFEDMLSLAERGDNTAVDMLVGDIYGSGYGKIGLKSTTIASSFGKVYKMKRQAEQEAED

At Epoch: 17000.00
RLGVIVGTGTNACYMEKLENCELWDGDDQEPRQVIVNTEWGAFGDNGVIDFVRSHYDWEVDEESLNPSHQKFEKMISGMYMGELARRVILRLAREHLIFNGRLSQKMKTAYAFKTKYISEIESDPKGCFDETRKVLAKLDQVGSDDDCQCLKLVVSRVSSRAAHLVSAAIATVLNKMKRPHTTVGVDGSVYRYHPKFHQLMEAKIAELTNPDYKFDLMLSEDGSGRGAALVAAV
At Epoch: 18000.00
QVDIGIDLGTANTLVYLRGHGIVMDEPSVVAVTRGSHTVLNDGAAVGLEAKKMLGKTSYSVDVIRPLREGVIANFPITEAMLRYFISRVKARRMFSQTRVVIAIPFGITHAEMKAVYNSTMRAGADKVHLIEETLAAGLGSGLRIDDPTANLVVDIGGGTTGISVISVADIAFGATVRCAGDHMTDAVSDFIRERYKLQIGQQTAEQLKIELGSALPQNEHAAMQIRGQGENGRPATIEVSADDVREALRAPLHKILRGIDWVLENTPPELSADLVDRGILVTGGGALLPRIDDLISDHTGLNVTVADDPLTCVARGAGAYLDTINWQRS
At Epoch: 19000.00
TKDMGIDLGTANTLVYSKGKGIVLREPSVVAINNLTKKPLAVGTEAKQMIGRTPGNIVAIRPLKDGVIADFDITQTMLKKFIEKITNKSAFTSPRIIVCFPSGVTEVERRAIEEATKQAGAREVVLMEEPMAAAIGAGLPVDEPTGSMIVDIGGGTTEVAIISLGGIVTSKSLRIAGDELDQAIIGYIKREYNLMIGERTSEQIKMEIGSAFKADEFEEEASMEIKGRDLISGLPKTVVVTESQIREALKEPVAAIIEAIKTTLEKTPPELAADIMDKGIMLAGGGALLKGLDALINHETHMPVHIAESPLDCVALGAGKALDKFDLIRQ
At Epoch: 20000.00
KSDIGIDLGTASVLVYIKGKGVVIQEP

At Epoch: 49000.00
PSPRPSISQTPIRTKLQLVDLAGSESVGMSGVSGAALWENSCINRSLSALSDVLGALAEQRPHVPYRNSKLTHLLQDSIGGDAKLLVMLCVSPTQRFLTESLQSLGFGSRARQI
At Epoch: 50000.00
KEYTFDGVFDQESNQKEVYEDVGKPVLKDVLQGYNGSILAYGQTGAGKTHSLLNSGMGVDGKPDPKQAGLLPRLVAALFVHVGADVKHVYTVEASMLQIYNEQVDCLLGDDREKAQGLQVTGKSEVKGLVWHKCKTPNELLQCFQKGRMNLVYAETKMNKSSSRSHAVFQIKVSKRPRALDKTGTKGGKVEMKATFGKLTVVDLAGSERIKKSGVTGTQLKEATNINSSLLSFGNIVQALAEKKKFIPYRDSKLTRILEDSVGGNCKTSLLVCCSPSAESSDETVSTLEFASRAARI
At Epoch: 51000.00
VEDMATLAQLHEGSIMHNLHIRYKKDNIYTYIGSILVSVNPYKSISGLYDITSMEQYSSYHLGERPPHIFAIANECYHCLWKRNDNQCVLISGESGAGKTESTKLILKYLSAMSQHSLDVTAKENVSYVEQAILESSPIMEVFGNAKTIHNNNSSRFGKFIKLNFCQKGNIQGGRIIDYLLEKNRVVRQNPGERNYHIFYALLAGTDEAQKEMFYLSEKENYYYLRQFGCIVDNAIDDQRTFQEVMTAMRVMKFSSEEILEVLKLLAGVLHLGNIEFVIAGGAQVSSKNALGRAAELLGLDSMKFTEVLTHRSMILRGEEISTPLTVEQGIDSRDSMAMALYSQCFSWIIKKINNRIKGKEDYRSVGVLDIFGFENFEVNRFEQFNINYANEKLQEYFNKHIFSLEQLEYNRDGLIWEDIDWMDNGECLDLIEKKMGILALINEESHFPKGTDDTLLAKLHSHHSKNPFYVKPRVLDHYFGVKHYAGEVLYHVKGILEKNRDTFRCDVLNLLCESRLDFIYDLFEHASSK